In [43]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

In [2]:
df = pickle.load(open('dataset.pkl', 'rb'))
df.head()

,Target_Vec_0,Target_Vec_1,Target_Vec_2,Target_Vec_3,Target_Vec_4,Target_Vec_5,Target_Vec_6,Target_Vec_7,Target_Vec_8,Target_Vec_9,...,Drug_Vec_91,Drug_Vec_92,Drug_Vec_93,Drug_Vec_94,Drug_Vec_95,Drug_Vec_96,Drug_Vec_97,Drug_Vec_98,Drug_Vec_99,Class
0,0.000013,-0.000028,0.000958,0.000242,0.000192,-0.000136,0.000343,0.000361,0.000666,0.001198,...,-0.027773,-0.261224,-0.349108,0.229779,0.161475,0.124695,0.368989,-0.198983,-0.435481,1
1,0.000013,-0.000028,0.000958,0.000242,0.000192,-0.000136,0.000343,0.000361,0.000666,0.001198,...,-0.009263,-0.166779,-0.276308,0.178456,0.128912,0.083457,0.357332,-0.184330,-0.500235,1
2,0.000013,-0.000028,0.000958,0.000242,0.000192,-0.000136,0.000343,0.000361,0.000666,0.001198,...,-0.009263,-0.166779,-0.276308,0.178456,0.128912,0.083457,0.357332,-0.184330,-0.500235,1
3,0.000013,-0.000028,0.000958,0.000242,0.000192,-0.000136,0.000343,0.000361,0.000666,0.001198,...,-0.197189,-0.190077,-0.227381,0.206817,0.102296,0.015936,0.315439,-0.296255,-0.259058,1
4,0.000013,-0.000028,0.000958,0.000242,0.000192,-0.000136,0.000343,0.000361,0.000666,0.001198,...,-0.197189,-0.190077,-0.227381,0.206817,0.102296,0.015936,0.315439,-0.296255,-0.259058,1


In [18]:
train_x, test_x, train_y, test_y = train_test_split(df.iloc[:,:-1].values, df.iloc[:,-1].values, train_size=0.8)

In [30]:
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=200))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(25, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [31]:
model.fit(train_x, train_y, epochs=10, batch_size=32)

Epoch 1/10
17580/17580 [==============================] - 4s 243us/step - loss: 0.6571 - acc: 0.6407
Epoch 2/10
17580/17580 [==============================] - 3s 171us/step - loss: 0.6553 - acc: 0.6416
Epoch 3/10
17580/17580 [==============================] - 3s 168us/step - loss: 0.6547 - acc: 0.6415
Epoch 4/10
17580/17580 [==============================] - 3s 176us/step - loss: 0.6543 - acc: 0.6415
Epoch 5/10
17580/17580 [==============================] - 3s 186us/step - loss: 0.6528 - acc: 0.6415
Epoch 6/10
17580/17580 [==============================] - 3s 174us/step - loss: 0.6530 - acc: 0.6415
Epoch 7/10
17580/17580 [==============================] - 3s 166us/step - loss: 0.6528 - acc: 0.6415
Epoch 8/10
17580/17580 [==============================] - 3s 166us/step - loss: 0.6521 - acc: 0.6415
Epoch 9/10
17580/17580 [==============================] - 3s 166us/step - loss: 0.6526 - acc: 0.6415
Epoch 10/10
17580/17580 [==============================] - 3s 168us/step - loss: 0.6542 - a

In [32]:
model.evaluate(test_x, test_y, batch_size=32)

4395/4395 [==============================] - 1s 116us/step


[0.6489434566242839, 0.6377701934558404]

In [33]:
pred_y = model.predict(test_x, batch_size=32)

In [36]:
pred_y = [1 if y >= 0.5 else 0 for y in pred_y]

In [37]:
np.unique(pred_y)

array([1])

In [39]:
lr = LogisticRegression()
lr.fit(train_x, train_y)
lr.score(test_x, test_y)

/home/sarthak/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6375426621160409

In [42]:
y_pred = lr.predict(test_x)

In [44]:
confusion_matrix(y_pred, test_y)

array([[  57,   58],
       [1535, 2745]])